<a href="https://colab.research.google.com/github/jbarrasa/goingmeta/blob/main/session5/Ontology_Driven_KG_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [166]:
pip install rdflib neo4j pandas


[notice] A new release of pip is available: 23.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [167]:
import rdflib, time
import pathlib
import pandas as pd

from neo4j import GraphDatabase, basic_auth

In [168]:
print("Britt settings:")
data_dir="https://raw.githubusercontent.com/jbarrasa/goingmeta/main/session5/data/"
data_dir=r"S:\Users\Britt\Documents\Visual_Studio_GitHub\Visual_Studio_Code_Python\GoingMeta\session5\data" + r"\\"
#print(data_dir)
print(pathlib.Path(data_dir))

onto = "https://raw.githubusercontent.com/jbarrasa/goingmeta/main/session5/ontos/rail.ttl"
onto = "S:/Users/Britt/Documents/Visual_Studio_GitHub/Visual_Studio_Code_Python/GoingMeta/session5/ontos/rail.ttl"
print(onto)

Britt settings:
S:\Users\Britt\Documents\Visual_Studio_GitHub\Visual_Studio_Code_Python\GoingMeta\session5\data
S:/Users/Britt/Documents/Visual_Studio_GitHub/Visual_Studio_Code_Python/GoingMeta/session5/ontos/rail.ttl


In [169]:
#utility function to get the local part of a URI (stripping out the namespace)
print("Verify rdf ontology functions getLocalPart and getNamespacePart")

def getLocalPart(uri):
  pos = -1
  pos = uri.rfind('#') 
  if pos < 0 :
    pos = uri.rfind('/')  
  if pos < 0 :
    pos = uri.rindex(':')
  return uri[pos+1:]

def getNamespacePart(uri):
  pos = -1
  pos = uri.rfind('#') 
  if pos < 0 :
    pos = uri.rfind('/')  
  if pos < 0 :
    pos = uri.rindex(':')
  return uri[0:pos+1]

# quick test
print(getLocalPart("http://onto.neo4j.com/rail#Station"))
print(getNamespacePart("http://onto.neo4j.com/rail#Station"))

Verify rdf ontology functions getLocalPart and getNamespacePart
Station
http://onto.neo4j.com/rail#


In [170]:
# processing the ontology...
print("Create graph from rdf ontology")
print(onto)

graph_onto = rdflib.Graph()
graph_onto.parse(onto, format='turtle')

graph_query_classes = """
prefix owl: <http://www.w3.org/2002/07/owl#> 
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
SELECT DISTINCT ?c
  WHERE {
    ?c rdf:type owl:Class .    
  } """
print("\nQuery graph for all owl:Class")
for row in graph_onto.query(graph_query_classes):
    print(str(row.c), getLocalPart(str(row.c)), getNamespacePart(str(row.c)))

graph_query_properties = """
prefix owl: <http://www.w3.org/2002/07/owl#> 
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
SELECT DISTINCT ?c
  WHERE {
    ?c rdf:type owl:ObjectProperty .    
  } """
print("\nQuery graph for all owl:ObjectProperty")
for row in graph_onto.query(graph_query_properties):
    print(str(row.c), getLocalPart(str(row.c)), getNamespacePart(str(row.c)))


Create graph from rdf ontology
S:/Users/Britt/Documents/Visual_Studio_GitHub/Visual_Studio_Code_Python/GoingMeta/session5/ontos/rail.ttl

Query graph for all owl:Class
http://onto.neo4j.com/rail#Event Event http://onto.neo4j.com/rail#
http://onto.neo4j.com/rail#Station Station http://onto.neo4j.com/rail#

Query graph for all owl:ObjectProperty
http://onto.neo4j.com/rail#affects affects http://onto.neo4j.com/rail#
http://onto.neo4j.com/rail#link link http://onto.neo4j.com/rail#


In [171]:
# read the onto and generate cypher (complete without mappings)
print("Generate cypher query from ontology (without mappings)")

classes_and_props_query = """
prefix owl: <http://www.w3.org/2002/07/owl#> 
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 

SELECT DISTINCT ?curi (GROUP_CONCAT(DISTINCT ?propTypePair ; SEPARATOR=",") AS ?props)
WHERE {
    ?curi rdf:type owl:Class .
    optional { 
      ?prop rdfs:domain ?curi ;
        a owl:DatatypeProperty ;
        rdfs:range ?range .
      BIND (concat(str(?prop),';',str(?range)) AS ?propTypePair)
    }
  } GROUP BY ?curi  """

cypher_list = []

for row in graph_onto.query(classes_and_props_query):
    cypher = []
    cypher.append("unwind $records AS record")
    cypher.append("merge (n:" + getLocalPart(row.curi) + " { `<id_prop>`: record.`<col with id>`} )")
    for pair in row.props.split(","):
      propName = pair.split(";")[0]
      propType = pair.split(";")[1]
      cypher.append("set n." + getLocalPart(propName) + " = record.`<col with value for " + getLocalPart(propName) + ">`")
    cypher.append("return count(*) as total") 
    cypher_list.append(' \n'.join(cypher))


rels_query = """
prefix owl: <http://www.w3.org/2002/07/owl#> 
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 

SELECT DISTINCT ?rel ?dom ?ran #(GROUP_CONCAT(DISTINCT ?relTriplet ; SEPARATOR=",") AS ?rels)
WHERE {
    ?rel a ?propertyClass .
    filter(?propertyClass in (rdf:Property, owl:ObjectProperty, owl:FunctionalProperty, owl:AsymmetricProperty, 
           owl:InverseFunctionalProperty, owl:IrreflexiveProperty, owl:ReflexiveProperty, owl:SymmetricProperty, owl:TransitiveProperty))
    
    ?rel rdfs:domain ?dom ;
      rdfs:range ?ran .
    
    #BIND (concat(str(?rel),';',str(?dom),';',str(?range)) AS ?relTriplet)
    
  }"""


for row in graph_onto.query(rels_query):
  cypher = []
  cypher.append("unwind $records AS record")
  cypher.append("match (source:" + getLocalPart(row.dom) + " { `<id_prop>`: record.`<col with source id>`} )")
  cypher.append("match (target:" + getLocalPart(row.ran) + " { `<id_prop>`: record.`<col with target id>`} )")
  cypher.append("merge (source)-[r:`"+ getLocalPart(row.rel) +"`]->(target)")
  cypher.append("return count(*) as total") 
  cypher_list.append(' \n'.join(cypher))

print("\nList of generated cypher queries")
for q in cypher_list:
  print("\n" + q)


Generate cypher query from ontology (without mappings)

List of generated cypher queries

unwind $records AS record 
merge (n:Event { `<id_prop>`: record.`<col with id>`} ) 
set n.eventDescription = record.`<col with value for eventDescription>` 
set n.eventId = record.`<col with value for eventId>` 
set n.eventType = record.`<col with value for eventType>` 
return count(*) as total

unwind $records AS record 
merge (n:Station { `<id_prop>`: record.`<col with id>`} ) 
set n.lat = record.`<col with value for lat>` 
set n.long = record.`<col with value for long>` 
set n.stationAddress = record.`<col with value for stationAddress>` 
set n.stationCode = record.`<col with value for stationCode>` 
set n.stationName = record.`<col with value for stationName>` 
return count(*) as total

unwind $records AS record 
match (source:Event { `<id_prop>`: record.`<col with source id>`} ) 
match (target:Station { `<id_prop>`: record.`<col with target id>`} ) 
merge (source)-[r:`affects`]->(target) 
ret

In [172]:
print("Define mappings: rail, station, event, link, affects")

railMappings = {}
stationMapping = {}
eventMapping = {}
linkMapping = {}
affectsMapping = {}

print(pathlib.Path(data_dir + "nr-stations-all.csv"))

stationMapping["@fileName"] = data_dir + "nr-stations-all.csv"
stationMapping["@uniqueId"] = "stationCode"
stationMapping["lat"] = "lat"
stationMapping["long"] = "long"
stationMapping["stationAddress"] = "address"
stationMapping["stationCode"] = "crs"
stationMapping["stationName"] = "name"
railMappings["Station"] = stationMapping

eventMapping["@fileName"] = data_dir + "nr-events.csv"
eventMapping["@uniqueId"] = "eventId"
eventMapping["eventDescription"] = "desc"
eventMapping["eventId"] = "id"
eventMapping["timestamp"] = "ts"
eventMapping["eventType"] = "type"
railMappings["Event"] = eventMapping

linkMapping["@fileName"] = data_dir + "nr-station-links.csv"
linkMapping["@from"] = "origin"
linkMapping["@to"] = "destination"
railMappings["link"] = linkMapping

affectsMapping["@fileName"] = data_dir + "nr-events.csv"
affectsMapping["@from"] = "id"
affectsMapping["@to"] = "Station"
railMappings["affects"] = affectsMapping

# show it?
print("\nExample mappings: railMappings")
railMappings

Define mappings: rail, station, event, link, affects
S:\Users\Britt\Documents\Visual_Studio_GitHub\Visual_Studio_Code_Python\GoingMeta\session5\data\nr-stations-all.csv

Example mappings: railMappings


{'Station': {'@fileName': 'S:\\Users\\Britt\\Documents\\Visual_Studio_GitHub\\Visual_Studio_Code_Python\\GoingMeta\\session5\\data\\\\nr-stations-all.csv',
  '@uniqueId': 'stationCode',
  'lat': 'lat',
  'long': 'long',
  'stationAddress': 'address',
  'stationCode': 'crs',
  'stationName': 'name'},
 'Event': {'@fileName': 'S:\\Users\\Britt\\Documents\\Visual_Studio_GitHub\\Visual_Studio_Code_Python\\GoingMeta\\session5\\data\\\\nr-events.csv',
  '@uniqueId': 'eventId',
  'eventDescription': 'desc',
  'eventId': 'id',
  'timestamp': 'ts',
  'eventType': 'type'},
 'link': {'@fileName': 'S:\\Users\\Britt\\Documents\\Visual_Studio_GitHub\\Visual_Studio_Code_Python\\GoingMeta\\session5\\data\\\\nr-station-links.csv',
  '@from': 'origin',
  '@to': 'destination'},
 'affects': {'@fileName': 'S:\\Users\\Britt\\Documents\\Visual_Studio_GitHub\\Visual_Studio_Code_Python\\GoingMeta\\session5\\data\\\\nr-events.csv',
  '@from': 'id',
  '@to': 'Station'}}

In [173]:
#copy of previous but using the mappings
print("Define getLoadersFromOnto to repeat process with mappings")

def getLoadersFromOnto(onto, rdf_format, mappings):
  graph_onto_mapped = rdflib.Graph()
  graph_onto_mapped.parse(onto, format= rdf_format)

  classes_and_props_query = """
  prefix owl: <http://www.w3.org/2002/07/owl#> 
  prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 

  SELECT DISTINCT ?curi (GROUP_CONCAT(DISTINCT ?propTypePair ; SEPARATOR=",") AS ?props)
  WHERE {
      ?curi rdf:type owl:Class .
      optional { 
        ?prop rdfs:domain ?curi ;
          a owl:DatatypeProperty ;
          rdfs:range ?range .
        BIND (concat(str(?prop),';',str(?range)) AS ?propTypePair)
      }
    } GROUP BY ?curi  """

  cypher_import = {}
  export_ns = set()
  export_mappings = {}

  for row in graph_onto_mapped.query(classes_and_props_query):
      export_ns.add(getNamespacePart(row.curi))
      export_mappings[getLocalPart(row.curi)] = str(row.curi)
      cypher = []
      cypher.append("unwind $records AS record")
      cypher.append("merge (n:" + getLocalPart(row.curi) + " { `" + mappings[getLocalPart(row.curi)]["@uniqueId"] + "`: record.`" + mappings[getLocalPart(row.curi)][mappings[getLocalPart(row.curi)]["@uniqueId"]] + "`} )")
      for pair in row.props.split(","):      
        propName = pair.split(";")[0]
        propType = pair.split(";")[1]
        export_ns.add(getNamespacePart(propName))
        export_mappings[getLocalPart(propName)] = propName
        #if a mapping (a column in the source file) is defined for the property and property is not a unique id
        if getLocalPart(propName) in mappings[getLocalPart(row.curi)] and getLocalPart(propName) != mappings[getLocalPart(row.curi)]["@uniqueId"]:
          cypher.append("set n." + getLocalPart(propName) + " = record.`" + mappings[getLocalPart(row.curi)][getLocalPart(propName)] + "`")
      cypher.append("return count(*) as total") 
      cypher_import[getLocalPart(row.curi)] = ' \n'.join(cypher)


  rels_query = """
  prefix owl: <http://www.w3.org/2002/07/owl#> 
  prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 

  SELECT DISTINCT ?rel ?dom ?ran #(GROUP_CONCAT(DISTINCT ?relTriplet ; SEPARATOR=",") AS ?rels)
  WHERE {
      ?rel a ?propertyClass .
      filter(?propertyClass in (rdf:Property, owl:ObjectProperty, owl:FunctionalProperty, owl:AsymmetricProperty, 
            owl:InverseFunctionalProperty, owl:IrreflexiveProperty, owl:ReflexiveProperty, owl:SymmetricProperty, owl:TransitiveProperty))
      
      ?rel rdfs:domain ?dom ;
        rdfs:range ?ran .
      
      #BIND (concat(str(?rel),';',str(?dom),';',str(?range)) AS ?relTriplet)
      
    }"""

  for row in graph_onto_mapped.query(rels_query):
    export_ns.add(getNamespacePart(row.rel))
    export_mappings[getLocalPart(row.rel)] = str(row.rel)
    cypher = []
    cypher.append("unwind $records AS record")
    cypher.append("match (source:" + getLocalPart(row.dom) + " { `" + mappings[getLocalPart(row.dom)]["@uniqueId"] + "`: record.`" + mappings[getLocalPart(row.rel)]["@from"] + "`} )")
    cypher.append("match (target:" + getLocalPart(row.ran) + " { `" + mappings[getLocalPart(row.ran)]["@uniqueId"] + "`: record.`" + mappings[getLocalPart(row.rel)]["@to"] + "`} )")
    cypher.append("merge (source)-[r:`"+ getLocalPart(row.rel) +"`]->(target)")
    cypher.append("return count(*) as total") 
    cypher_import[getLocalPart(row.rel)] = ' \n'.join(cypher)


  nscount = 0
  mapping_export_cypher = []
  
  for ns in export_ns:
    mapping_export_cypher.append("call n10s.nsprefixes.add('ns" + str(nscount) + "','" + ns + "');")
    nscount+=1

  for k in export_mappings.keys():
    mapping_export_cypher.append("call n10s.mapping.add('" + export_mappings[k] + "','" + k + "');")

  return cypher_import ,  mapping_export_cypher




Define getLoadersFromOnto to repeat process with mappings


In [174]:
print("Run getLoadersFromOnto")
print("using ontology " + onto)
cypher_import , mapping_defs = getLoadersFromOnto(onto,"turtle",railMappings)

print("#LOADERS:\n")
for q in cypher_import.keys():
  print(q + ": \n\nfile: " + railMappings[q]["@fileName"] + "\n\n"+ cypher_import[q] + "\n\n")

print("#EXPORT MAPPINGS (for RDF API):\n")
for md in mapping_defs:
  print(md)

Run getLoadersFromOnto
using ontology S:/Users/Britt/Documents/Visual_Studio_GitHub/Visual_Studio_Code_Python/GoingMeta/session5/ontos/rail.ttl
#LOADERS:

Event: 

file: S:\Users\Britt\Documents\Visual_Studio_GitHub\Visual_Studio_Code_Python\GoingMeta\session5\data\\nr-events.csv

unwind $records AS record 
merge (n:Event { `eventId`: record.`id`} ) 
set n.eventDescription = record.`desc` 
set n.eventType = record.`type` 
return count(*) as total


Station: 

file: S:\Users\Britt\Documents\Visual_Studio_GitHub\Visual_Studio_Code_Python\GoingMeta\session5\data\\nr-stations-all.csv

unwind $records AS record 
merge (n:Station { `stationCode`: record.`crs`} ) 
set n.lat = record.`lat` 
set n.long = record.`long` 
set n.stationAddress = record.`address` 
set n.stationName = record.`name` 
return count(*) as total


affects: 

file: S:\Users\Britt\Documents\Visual_Studio_GitHub\Visual_Studio_Code_Python\GoingMeta\session5\data\\nr-events.csv

unwind $records AS record 
match (source:Event {

In [175]:
# Utility function to write to Neo4j in batch mode.
print("Define insert_data to batch insert to the graph")

def insert_data(session, query, frame, batch_size = 500):

    total = 0
    batch = 0
    start = time.time()
    result = None
    
    while batch * batch_size < len(frame):
        res = session.write_transaction( lambda tx: tx.run(query,
                      parameters = {'records': frame[batch*batch_size:(batch+1)*batch_size].to_dict('records')}).data())

        total += res[0]['total']
        batch += 1
        result = {"total":total, 
                  "batches":batch, 
                  "time":time.time()-start}
        print(result)
        
    return result

Define insert_data to batch insert to the graph


This section is for Britt's CSU DevTest code

In [191]:
print("DevTest code")

print("Trying to connect to my free neo4j Aura instance: Britt/CSU")
neo4j_db_uri="neo4j+s://5ba77883.databases.neo4j.io"
print(neo4j_db_uri)

# driver = GraphDatabase.driver(
#   "bolt://3.84.57.61:7687",
#   auth=basic_auth("neo4j", "south-kettles-harmony"))
driver = GraphDatabase.driver(
  neo4j_db_uri,
  auth=basic_auth("neo4j", "QWWDLjevJTXdWnmZqjXJFENM6_4k660WxN3aqr2SLSg"))

print(driver.verify_authentication())
print(driver.verify_connectivity())


session = driver.session(database="CSU_01")
print(driver.verify_connectivity())
session = driver.session(database="5ba77883")
print(driver.verify_connectivity())

print("Created driver session")

# print("Results from previous getLoadersFromOnto")
# print("#LOADERS:\n")
# for q in cypher_import.keys():
#   print(q + ": \n\nfile: " + railMappings[q]["@fileName"] + "\n\n"+ cypher_import[q] + "\n\n")

# print("#EXPORT MAPPINGS (for RDF API):\n")
# for md in mapping_defs:
#   print(md)


DevTest code
Trying to connect to my free neo4j Aura instance: Britt/CSU
neo4j+s://5ba77883.databases.neo4j.io
True
None
None
None
Created driver session


In [182]:
for q in cypher_import.keys():
  print("about to import " + q + " from file " + railMappings[q]["@fileName"])
  df = pd.read_csv(railMappings[q]["@fileName"])
  result = insert_data(session, cypher_import[q], df, batch_size = 300) 
  print(result)
  
  
  #load csv from 'file:///artists.csv' AS row
  #LOAD CSV from railMappings[q]["@fileName"] AS row
  
  
  #print("How do you create a data_frame from a .csv file?")
  #insert_data(session, cypher_import[q], data_frame, batch_size=300)

about to import Event from file S:\Users\Britt\Documents\Visual_Studio_GitHub\Visual_Studio_Code_Python\GoingMeta\session5\data\\nr-events.csv


C:\Users\Britt\AppData\Local\Temp\ipykernel_11744\2252819377.py:12: DeprecationWarning: write_transaction has been renamed to execute_write
  res = session.write_transaction( lambda tx: tx.run(query,


ClientError: {code: Neo.ClientError.Database.DatabaseNotFound} {message: Unable to get a routing table for database '5ba77883' because this database does not exist}

In [183]:
print("Closing session and driver")
session.close()
driver.close()

Closing session and driver


In [47]:
print("Connect to a neo4j sandbox")
driver = GraphDatabase.driver(
  "bolt://3.84.57.61:7687",
  auth=basic_auth("neo4j", "south-kettles-harmony"))

session = driver.session(database="neo4j")

#cypher_import , mapping_defs = getLoadersFromOnto(onto,"turtle",railMappings)

for q in cypher_import.keys():
  print("about to import " + q + " from file " + railMappings[q]["@fileName"])
  df = pd.read_csv(railMappings[q]["@fileName"])
  result = insert_data(session, cypher_import[q], df, batch_size = 300) 
  print(result)

for md in mapping_defs:
  session.run(md)
  

ModuleNotFoundError: No module named 'pandas'